## Importing Libraries

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os


In [6]:
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = r"D:\DigiMark\Exracting Word Via Images\src\data\templates\invoice.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
}

payload = {
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "system",
      "content": "You are an experienced mobile developer, a web scraper, and an expert in image processing and generative AI. You will be provided with an image, which could be a screenshot of a document or PDF, or it could contain handwritten text. Your task is to extract all of the relevant data from the image and return it in a specified format."
    },
    {
      "role": "user",
      "content": [{  
                    "type":"text",
                    "text":"""
What: 
Please help me extract the textual pieces of information from the image provided. The image you will receive is that of and invoice, it can and will have the following as its content:
Title of Invoice (this could be ‘Cash Sale’, ‘Product Purchase’ or ‘Invoice’)
Date (Convert it to “DD/MM/YYYY” format) 	
Reference Number (Ref No)	
Customer Details (this will contain the following details. I.e, name, address, phone number, email, credit terms)
Business Details (this will contain the following details. I.e, name, address, phone number, email)
Data in tabular format which will have the following:
Item
Price(USD)
Quantity or Qty
Add Tax
Total (USD)

Your job is to extract all of the aforementioned from the image.The desired output I want to receive is in JSON format and just this nothing else. If you are unable to find any of the aforementioned details in the image please add ‘N/A’ in the json with the respective key.

Who:
 I am a professional Mobile application developer in Flutter.

Why:
This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.
Which:
You have to extract everything specified as before.As for the tabular part please observe the column names of the table as they are supposed to be key names in the JSON also populate each key with data provided with respect to each column.The data you are supposed to extract from the image is 'Item', 'Price(USD), 'Quantity', 'Add Tax', 'Total (USD)', Further more in the table there is also a part which contains sections named ‘Sub Total’, ‘Tax’, ‘Total’, please also retrieve these. Please note that the wording of the column names may differ, please  extract them according to the context.Following is the template on how you should create the JSON:
{
  "title_of_invoice": "<value>",
  "date": "<value>",
  "ref_no": "<value>",
  "customer_details": {
    "name": "<value>",
    "address": "<value>",
    "phone_number": "<value>",
    "email": "<value>",
    "credit_terms": "<value>"
  },
  "business_details": {
    "name": "<value>",
    "address": "<value>",
    "phone_number": "<value>",
    "email": "<value>"
  },
  "item_list": [
    {
      "item": "<value>",
      "price_usd": "<value>",
      "quantity": "<value>",
      "add_tax": "<value>"
    }
  ],
  "total_prices": {
    "sub_total": "<value>",
    "tax": "<value>",
    "total": "<value>"
  }
}
How: 
Finally, return the data as a JSON response and only JSON and nothing else.


"""
                  },
                   {
                      "type": "image_url",
                      "image_url": {
                      "url": f"data:image/png;base64,{base64_image}"
                      }                     
                   }
      ]
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9G18DDK9QomI360Rb3APAOYoHfPt9', 'object': 'chat.completion', 'created': 1713604625, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "title_of_invoice": "Invoice",\n  "date": "12/04/2024",\n  "ref_no": "INV-04-001",\n  "customer_details": {\n    "name": "John Smith",\n    "address": "5678 Oak Avenue Portland, OR 97201",\n    "phone_number": "(503) 555-6789",\n    "email": "jsmith@gmail.com",\n    "credit_terms": "30 Days"\n  },\n  "business_details": {\n    "name": "Auto Parts Store",\n    "address": "1234 Maple Street Springfield, IL 62701",\n    "phone_number": "(217) 555-1234",\n    "email": "info@autopartsstore.com"\n  },\n  "item_list": [\n    {\n      "item": "Engine Oil (5W-30)",\n      "price_usd": "25.00",\n      "quantity": "3",\n      "add_tax": "8%"\n    },\n    {\n      "item": "Brake Pads (Front)",\n      "price_usd": "50.00",\n      "quantity": "1",\n      "add_tax": "8%"\n    },\n    {\n      

In [7]:
print(response.json()['choices'][0]['message']['content'])

{
  "title_of_invoice": "Invoice",
  "date": "12/04/2024",
  "ref_no": "INV-04-001",
  "customer_details": {
    "name": "John Smith",
    "address": "5678 Oak Avenue Portland, OR 97201",
    "phone_number": "(503) 555-6789",
    "email": "jsmith@gmail.com",
    "credit_terms": "30 Days"
  },
  "business_details": {
    "name": "Auto Parts Store",
    "address": "1234 Maple Street Springfield, IL 62701",
    "phone_number": "(217) 555-1234",
    "email": "info@autopartsstore.com"
  },
  "item_list": [
    {
      "item": "Engine Oil (5W-30)",
      "price_usd": "25.00",
      "quantity": "3",
      "add_tax": "8%"
    },
    {
      "item": "Brake Pads (Front)",
      "price_usd": "50.00",
      "quantity": "1",
      "add_tax": "8%"
    },
    {
      "item": "Air Filter (Toyota Camry)",
      "price_usd": "15.00",
      "quantity": "2",
      "add_tax": "8%"
    },
    {
      "item": "Spark Plugs (Set of 4)",
      "price_usd": "20.00",
      "quantity": "1",
      "add_tax": "8%"
 

In [6]:
import json

# JSON response string
json_response = "{\n\t\"last_balance_sheet_date\": \"January 31, 2024\",\n\t\"cash_on_hand\": \"$10,000\",\n\t\"bank_name\": \"First National Bank\",\n\t\"bank_balance\": \"$50,000\",\n\t\"list_of_products\": [\n\t\t{\n\t\t\t\"product_name\": \"Widget A\",\n\t\t\t\"product_cost\": \"$10\",\n\t\t\t\"quantity\": \"100\",\n\t\t\t\"delivery_charges\": \"$5\"\n\t\t},\n\t\t{\n\t\t\t\"product_name\": \"Widget B\",\n\t\t\t\"product_cost\": \"$20\",\n\t\t\t\"quantity\": \"50\",\n\t\t\t\"delivery_charges\": \"$8\"\n\t\t},\n\t\t{\n\t\t\t\"product_name\": \"Widget C\",\n\t\t\t\"product_cost\": \"$15\",\n\t\t\t\"quantity\": \"75\",\n\t\t\t\"delivery_charges\": \"$6\"\n\t\t}\n\t]\n}"

# Parse JSON string
data = json.loads(json_response)

# Write formatted JSON data to a file
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)

In [10]:

import os
import config
from utils import encode_image, extract_file_type
import requests
from dotenv import load_dotenv
import json
load_dotenv()


file_path = 'data/bank_sheet.png'


base64_image = encode_image(file_path)

headers = {
"Content-Type": "application/json",
"Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
}

payload = {
    'model':"gpt-4-turbo",
    'messages': config.PROMPT + [{
    "role": "user",
    "content": [{
        "type": "image_url",
        "image_url": {
            "url": f"data:image/" + extract_file_type(file_path) + f";base64,{base64_image}"
        }
    }]
}],
    'max_tokens':1000,
}
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

with open('../output.json', 'w') as f:
    json.dump(response.json()['choices'][0]['message']['content'], f, indent=4)


In [9]:
import json

json_response = response.json()['choices'][0]['message']['content']

data = json.loads(json_response)

with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)


In [ ]:
"""
What: Please help me extract the textual pieces of information from the image provided. The image you will receive will have data in tabular format, your job is to focus on the tabular data and ignore everything else.The desired output I want to receive is in JSON format and just this nothing else.

Who: I am a professional Mobile application developer in Flutter.

Why: This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.

Which: Please observe the column names of the table as they are supposed to be key names in the JSON also populate each key with data provided with respect to each column.The data you are supposed to extract from the image is 'Item', 'Price(USD), 'Quantity', 'Add Tax', 'Total (USD)'. Please note that the wording of the column names may differ, please  extract them according to the context.Following is the template on how you should create the JSON:
{
{'Item':, 
'Price(USD):, 
'Quantity', 
'Add Tax',
 'Total (USD)
}
}

How: Finally, return the data as a JSON response and only JSON nothing else.
"""
